# FINAL OUTPUT 

In [33]:
# Aleksandra

# importing Sales data into SQL 

# Importing necessary libraries:
# Importing the required libraries, which are pandas for data manipulation and psycopg2 for connecting to the PostgreSQL database.
import pandas as pd
import psycopg2


# Reading data from Excel:
# Using the pd.read_excel function from the pandas library to read data from an Excel file named 'sales_cleaned_NEW.xlsx'. This file contains information about movie sales.
movies_data = pd.read_excel(r'C:\Users\memwa\Desktop\cleaned_data\sales_cleaned_NEW.xlsx') # cleaned Sales file


# Establishing a database connection
# Establishing a connection to a PostgreSQL database using the psycopg2 library. The parameters for the connection include: host, port, database, user, password
conn = psycopg2.connect(
    host='localhost',
    port = 5433, 
    database='DBmovies',
    user='postgres',
    password='alex2000'
)

# Creating a cursor. A cursor is used to execute SQL commands and interact with the database
cursor = conn.cursor()

# Looping through each row of the movies_data dataframe using a for loop. This dataframe contains information about movie titles, release dates, production budgets, and worldwide box office earnings.
for _, movie_row in movies_data.iterrows():
    movie_title = movie_row['title']
    rel_date = movie_row['release_date']
    production_budget = movie_row['production_budget']
    worldwide_box_office = movie_row['worldwide_box_office ']

    # Inserting data into the 'movies' table:
    # Inside the loop, I extract specific columns (movie title, release date, production budget, worldwide box office) from the current row.
    # Execute an SQL INSERT statement to insert the mentioned data into the 'movies' table. The SQL query specifies the columns where data should be inserted (movie_title, rel_date, production_budget, worldwide_box_office), and I provide corresponding values from the dataframe row.
    cursor.execute(
        "INSERT INTO movies (movie_title, rel_date, production_budget, worldwide_box_office) "
        "VALUES (%s, %s, %s, %s)",
        (movie_title, rel_date, production_budget, worldwide_box_office)
    )

# Committing changes. After inserting data for each movie, I commit the changes to the database using conn.commit(). This ensures that the data is permanently saved in the 'movies' table
# Finally, I close the cursor and the database connection using cursor.close() and conn.close(), respectively. This is important for releasing resources and ensuring the connection is properly closed.
conn.commit()
cursor.close()
conn.close()

In [34]:
# Aleksandra

#Genre connection

# Importing necessary libraries
# Importing the required libraries, which are pandas for data manipulation and psycopg2 for connecting to the PostgreSQL database
import pandas as pd
import psycopg2  

# Reading data from Excel
# Using the pd.read_excel function from the pandas library to read data from an Excel file named 'metaclean_cleaned.xlsx'. This file contains information about movie and their associated genres
genre_data = pd.read_excel(r"C:\Users\memwa\Desktop\cleaned_data\metaclean_cleaned.xlsx")  

# Establishing a database connection
# Establishing a connection to a PostgreSQL database using the psycopg2 library. The parameters for the connection include: host, port, database, user, password
conn = psycopg2.connect(
    host='localhost',
    database='DBmovies',
    port = 5433,
    user='postgres',
    password='alex2000'
)

# Creating a cursor. A cursor is used to execute SQL commands and interact with the database
cursor = conn.cursor()

# Creating an empty dictionary called genre_id_mapping. This dictionary will be used to map genre names to genre IDs, ensuring that there are no duplicated genre entries in the database
genre_id_mapping = {}

# Looping through each row of the genre_data dataframe, which contains information about movie titles and associated genres
# Inside the loop, I extract the movie title and genre data from the current row
for _, genre_row in genre_data.iterrows():
    genre_names = genre_row['genre'].split(',')  # Genres are comma-separated, so I split them into a list
    movie_title = genre_row['title']  

    # Executing an SQL query to get the movie_id for the current movie title from the 'movies' table
    cursor.execute("SELECT movie_id FROM movies WHERE movie_title = %s", (movie_title,))
    movie_id_result = cursor.fetchone()

    if movie_id_result is not None:
        movie_id = movie_id_result[0]

        # I iterate through each genre in the list of genres associated with a movie
        for genre_name in genre_names:
            genre_name = genre_name.strip()  # Remove leading/trailing spaces

            # Checking if the genre name is already in the genre_id_mapping dictionary. If it is, I retrieve the corresponding genre ID; otherwise, the genre will be inserted into the 'genres' table with a corresponding genre ID in return
            if genre_name in genre_id_mapping:
                genre_id = genre_id_mapping[genre_name]
            else:
                cursor.execute("INSERT INTO genres (genre_name) VALUES (%s) RETURNING genre_id", (genre_name,))
                genre_id = cursor.fetchone()[0]
                genre_id_mapping[genre_name] = genre_id

                # Now, there is a genre_id for each genre type  

            # Checking if the movie-genre mapping already exists in the 'moviegenrelink' table
            cursor.execute("SELECT 1 FROM moviegenrelink WHERE movie_id = %s AND genre_id = %s", (movie_id, genre_id))
            existing_record = cursor.fetchone()

            if not existing_record:
                # Inserting movie-genre mapping - If there is no existing record, I insert the mapping into the 'moviegenrelink' junction table
                cursor.execute("INSERT INTO moviegenrelink (movie_id, genre_id) VALUES (%s, %s)", (movie_id, genre_id))
                # After inserting genre mappings for each movie, I commit the changes to the database using conn.commit(). This ensures that the data is permanently saved
                conn.commit()

# Finally, I close the cursor and the database connection using cursor.close() and conn.close() respectively, which is important for resource management and proper closure of the connection
cursor.close()
conn.close()

In [35]:
# Aleksandra

# Actors connection

# Importing necessary libraries
# Importing the required libraries, which are pandas for data manipulation and psycopg2 for connecting to the PostgreSQL database
import pandas as pd
import psycopg2 

# Reading data from Excel
# Using the pd.read_excel function from the pandas library to read data from an Excel file named 'metaclean_cleaned.xlsx'. This file contains information about movie and their associated actors
actor_data = pd.read_excel(r'C:\Users\memwa\Desktop\cleaned_data\metaclean_cleaned.xlsx')

# Establishing a database connection
# Establishing a connection to a PostgreSQL database using the psycopg2 library. The parameters for the connection include: host, port, database, user, password
conn = psycopg2.connect(
    host='localhost',
    database='DBmovies',
    port = 5433,
    user='postgres',
    password='alex2000'
)

# Creating a cursor. A cursor is used to execute SQL commands and interact with the database
cursor = conn.cursor()

# Creating an empty dictionary called actor_id_mapping. This dictionary will be used to map actors names to actors IDs, ensuring that there are no duplicated actor entries in the database
actor_id_mapping = {}

# Looping through each row of the actor_data dataframe, which contains information about movie titles and associated actors
# Inside the loop, I extract the movie title and actors data from the current row
for _, actor_row in actor_data.iterrows():
    actor_names = actor_row['cast'].split(',')  # Actors are comma-separated, so I split them into a list
    movie_title = actor_row['title'] 

    # Executing an SQL query to get the movie_id for the current movie title from the 'movies' table.
    cursor.execute("SELECT movie_id FROM movies WHERE movie_title = %s", (movie_title,))
    movie_id_result = cursor.fetchone()

    if movie_id_result is not None:
        movie_id = movie_id_result[0]

        # Iterating for each actor in the list of actors associated with a movie
        for actor_name in actor_names:
            actor_name = actor_name.strip()  # Remove leading/trailing spaces

            # Checking if the actor name is already in the actor_id_mapping dictionary. If it is, I retrieve the corresponding actor ID; otherwise, the actor will be inserted into the 'actors' table with a corresponding actor ID in return
            if actor_name in actor_id_mapping:
                actor_id = actor_id_mapping[actor_name]
            else:
                # If not, insert the actor into the 'actors' table and retrieve the actor_id
                cursor.execute("INSERT INTO actors (actor_name) VALUES (%s) RETURNING actor_id", (actor_name,))
                actor_id = cursor.fetchone()[0]
                actor_id_mapping[actor_name] = actor_id

            # Now, there is an actor_id for each actor 

            # Checking if the movie-actor mapping already exists in the 'actedin' table
            cursor.execute("SELECT 1 FROM actedin WHERE movie_id = %s AND actor_id = %s", (movie_id, actor_id))
            existing_record = cursor.fetchone()

            if not existing_record:
                # Inserting movie-actor mapping - If there is no existing record, I insert the mapping into the 'actedin' junction table
                cursor.execute("INSERT INTO actedin (movie_id, actor_id) VALUES (%s, %s)", (movie_id, actor_id))
                # After inserting genre mappings for each movie, I commit the changes to the database using conn.commit(). This ensures that the data is permanently saved
                conn.commit()

# Finally, I close the cursor and the database connection using cursor.close() and conn.close() respectively, which is important for resource management and proper closure of the connection
cursor.close()
conn.close()

In [37]:
# Aleksandra

# Studios connection

# Importing necessary libraries
# Importing the required libraries, which are pandas for data manipulation and psycopg2 for connecting to the PostgreSQL database
import psycopg2
import pandas as pd

# Establishing a database connection
# Establishing a connection to a PostgreSQL database using the psycopg2 library. The parameters for the connection include: host, port, database, user, password
conn = psycopg2.connect(
    host='localhost',
    database='DBmovies',
    port = 5433,
    user='postgres',
    password='alex2000'
)
# Creating a cursor. A cursor is used to execute SQL commands and interact with the database
cursor = conn.cursor()

# Reading data from Excel
# Using the pd.read_excel function from the pandas library to read data from an Excel file named 'metaclean_cleaned.xlsx'. This file contains information about movie titles and their associated studios
studio_data = pd.read_excel(r'C:\Users\memwa\Desktop\cleaned_data\metaclean_cleaned.xlsx') 

# Creating an empty dictionary called studio_id_mapping. This dictionary will be used to map studio names to studio IDs, ensuring that there are no duplicated studio entries in the database
studio_id_mapping = {}

# Looping through each row of the studio_data dataframe, which contains information about movie titles and associated studio 
# Inside the loop, I extract the movie title and studio data from the current row
for _, studio_row in studio_data.iterrows():
    movie_title = studio_row['title']  

    # Executing an SQL query to get the movie_id for the current movie title from the 'movies' table
    cursor.execute("SELECT movie_id FROM movies WHERE movie_title = %s", (movie_title,))
    movie_id_result = cursor.fetchone()

    if movie_id_result is not None:
        movie_id = movie_id_result[0]

        # Retrieving the studio name from the data
        studio_name = studio_row['studio']

        # Cleaning the studio name
        studio_name = studio_name.strip()

        # Checking if the studio name is already in the studio_id_mapping dictionary. If it is, I retrieve the corresponding studio ID; otherwise, the studio name will be inserted into the 'studios' table with a corresponding studio ID in return
        if studio_name in studio_id_mapping:
            studio_id = studio_id_mapping[studio_name]
        else:
            # If not, insert the studio into the 'studios' table and retrieve the studios_id
            cursor.execute("INSERT INTO studios (studio_name) VALUES (%s) RETURNING studio_id", (studio_name,))
            studio_id = cursor.fetchone()[0]
            studio_id_mapping[studio_name] = studio_id

        # Now, there is a studio_id for each studio

        # Checking if the movie-studio mapping already exists in the 'producedby' table
        cursor.execute("SELECT 1 FROM producedby WHERE movie_id = %s AND studio_id = %s", (movie_id, studio_id))
        existing_record = cursor.fetchone()

        if not existing_record:
            # Insert the mapping into the 'producedby' junction table
            cursor.execute("INSERT INTO producedby (movie_id, studio_id) VALUES (%s, %s)", (movie_id, studio_id))
            # After inserting studio mappings for each movie, I commit the changes to the database using conn.commit(). This ensures that the data is permanently saved
            conn.commit()

# Finally, I close the cursor and the database connection using cursor.close() and conn.close() respectively, which is important for resource management and proper closure of the connection
cursor.close()
conn.close()

In [38]:
# Aleksandra 

# Rating connection

# Importing necessary libraries
# Importing the required libraries, which are pandas for data manipulation and psycopg2 for connecting to the PostgreSQL database
import psycopg2
import pandas as pd

# Establishing a database connection
# Establishing a connection to a PostgreSQL database using the psycopg2 library. The parameters for the connection include: host, port, database, user, password
conn = psycopg2.connect(
    host='localhost',
    database='DBmovies',
    port = 5433,
    user='postgres',
    password='alex2000'
)

# Creating a cursor. A cursor is used to execute SQL commands and interact with the database
cursor = conn.cursor()

# Reading data from Excel
# Using the pd.read_excel function from the pandas library to read data from an Excel file named 'metaclean_cleaned.xlsx'. This file contains information about movie titles and their associated ratings
rating_data = pd.read_excel(r'C:\Users\memwa\Desktop\cleaned_data\metaclean_cleaned.xlsx')

# Creating an empty dictionary called rating_id_mapping. This dictionary will be used to map rating names to rating IDs, ensuring that there are no duplicated rating entries in the database
rating_id_mapping = {}

# Looping through each row of the rating_data dataframe, which contains information about movie titles and associated rating 
# Inside the loop, I extract the movie title and rating data from the current row
for _, rating_row in rating_data.iterrows():
    movie_title = rating_row['title'] 

    # Executing an SQL query to get the movie_id for the current movie title from the 'movies' table
    cursor.execute("SELECT movie_id FROM movies WHERE movie_title = %s", (movie_title,))
    movie_id_result = cursor.fetchone()

    if movie_id_result is not None:
        movie_id = movie_id_result[0]

        # Retrieving the rating name from the data
        rating_name = rating_row['rating']

        # Cleaning the rating name
        rating_name = rating_name.strip()

        # Checking if the rating name is already in the rating_id_mapping dictionary. If it is, I retrieve the corresponding studio ID; otherwise, the rating name will be inserted into the 'ratings' table with a corresponding rating ID in return
        if rating_name in rating_id_mapping:
            rating_id = rating_id_mapping[rating_name]
        else:
            # If not, insert the rating into the 'ratings' table and retrieve the rating_id
            cursor.execute("INSERT INTO ratings (rating_name) VALUES (%s) RETURNING rating_id", (rating_name,))
            rating_id = cursor.fetchone()[0]
            rating_id_mapping[rating_name] = rating_id

            # Now, there is a rating_id for each rating

        # Checking if the movie-rating mapping already exists in the 'movierating' table
        cursor.execute("SELECT 1 FROM movierating WHERE movie_id = %s AND rating_id = %s", (movie_id, rating_id))
        existing_record = cursor.fetchone()

        if not existing_record:
            # Insert the mapping into the 'movierating' junction table
            cursor.execute("INSERT INTO movierating (movie_id, rating_id) VALUES (%s, %s)", (movie_id, rating_id))
            # After inserting rating mappings for each movie, I commit the changes to the database using conn.commit(). This ensures that the data is permanently saved
            conn.commit()

# Finally, I close the cursor and the database connection using cursor.close() and conn.close() respectively, which is important for resource management and proper closure of the connection
cursor.close()
conn.close()

In [40]:
# Aleksandra 

# Directors connection

# Importing necessary libraries
# Importing the required libraries, which are pandas for data manipulation and psycopg2 for connecting to the PostgreSQL database
import psycopg2
import pandas as pd

# Establishing a database connection
# Establishing a connection to a PostgreSQL database using the psycopg2 library. The parameters for the connection include: host, port, database, user, password
conn = psycopg2.connect(
    host='localhost',
    database='DBmovies',
    port = 5433,
    user='postgres',
    password='alex2000'
)

# Creating a cursor. A cursor is used to execute SQL commands and interact with the database
cursor = conn.cursor()

# Reading data from Excel
# Using the pd.read_excel function from the pandas library to read data from an Excel file named 'metaclean_cleaned.xlsx'. This file contains information about movie titles and their associated directors
directors_data = pd.read_excel(r'C:\Users\memwa\Desktop\cleaned_data\metaclean_cleaned.xlsx')

# Creating an empty dictionary called rating_id_mapping. This dictionary will be used to map rating names to rating IDs, ensuring that there are no duplicated rating entries in the database
directors_id_mapping = {}

# Looping through each row of the director_data dataframe, which contains information about movie titles and associated director 
# Inside the loop, I extract the movie title and director data from the current row
for _, director_row in directors_data.iterrows():
    movie_title = director_row['title'] 

    # Executing an SQL query to get the movie_id for the current movie title from the 'movies' table
    cursor.execute("SELECT movie_id FROM movies WHERE movie_title = %s", (movie_title,))
    movie_id_result = cursor.fetchone()

    if movie_id_result is not None:
        movie_id = movie_id_result[0]

        # Retrieving the director name from the data
        director_name = director_row['director']

        # Cleaning the director name
        director_name = director_name.strip()

        # Checking if the director name is already in the director_id_mapping dictionary. If it is, I retrieve the corresponding director ID; otherwise, the director name will be inserted into the 'directors' table with a corresponding director ID in return
        if director_name in directors_id_mapping:
            director_id = directors_id_mapping[director_name]
        else:
            # If not, insert the director into the 'directors' table and retrieve the director_id
            cursor.execute("INSERT INTO directors (director_name) VALUES (%s) RETURNING director_id", (director_name,))
            director_id = cursor.fetchone()[0]
            directors_id_mapping[director_name] = director_id

            # Now, there is a director_id for each director

        # Checking if the movie-director mapping already exists in the 'directedby' table
        cursor.execute("SELECT 1 FROM directedby WHERE movie_id = %s AND director_id = %s", (movie_id, director_id))
        existing_record = cursor.fetchone()

        if not existing_record:
            # Insert the mapping into the 'directedby' junction table
            cursor.execute("INSERT INTO directedby (movie_id, director_id) VALUES (%s, %s)", (movie_id, director_id))
            # After inserting director mappings for each movie, I commit the changes to the database using conn.commit(). This ensures that the data is permanently saved
            conn.commit()

# # Finally, I close the cursor and the database connection using cursor.close() and conn.close() respectively, which is important for resource management and proper closure of the connection
cursor.close()
conn.close()

In [42]:
# Aleksandra 

# Genre JOIN

# Importing necessary libraries
# Importing the required libraries, which are pandas for data manipulation and psycopg2 for connecting to the PostgreSQL database
import psycopg2
import pandas as pd

# Establishing a database connection
# Establishing a connection to a PostgreSQL database using the psycopg2 library. The parameters for the connection include: host, port, database, user, password
conn = psycopg2.connect(
    host='localhost',
    database='DBmovies',
    port = 5433,
    user='postgres',
    password='alex2000'
)

# Creating a cursor. A cursor is used to execute SQL commands and interact with the database
cursor = conn.cursor()

# Defining the SQL query
sql_query = """
    SELECT genres.genre_name, SUM(movies.worldwide_box_office) AS total_worldwide_box_office
    FROM genres
    JOIN moviegenrelink ON genres.genre_id = moviegenrelink.genre_id
    JOIN movies ON movies.movie_id = moviegenrelink.movie_id
    GROUP BY genres.genre_name
    ORDER BY total_worldwide_box_office DESC;
"""

# Executing the query
cursor.execute(sql_query)

# Fetching all the rows
rows = cursor.fetchall()

# Creating a dataframe
df = pd.DataFrame(rows, columns=["Genre Name", "Total Worldwide Box Office"])

# Displaing the dataframe
print(df)

# Closing the cursor and database connection
cursor.close()
conn.close()



     Genre Name  Total Worldwide Box Office
0     Adventure                1.325439e+11
1        Action                1.228251e+11
2        Comedy                1.224680e+11
3         Drama                1.211516e+11
4      Thriller                1.121768e+11
5        Family                7.811698e+10
6       Fantasy                7.385427e+10
7        Sci-Fi                6.527696e+10
8     Animation                5.550553e+10
9       Romance                4.947243e+10
10        Crime                4.593611e+10
11      Mystery                3.805830e+10
12       Horror                2.575286e+10
13      Musical                1.716690e+10
14    Biography                1.565332e+10
15      History                1.148733e+10
16          War                1.112083e+10
17        Music                9.417470e+09
18        Sport                7.536724e+09
19      Western                3.380669e+09
20  Documentary                2.390551e+09
21        Adult                5